In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydrant.topology.geom as gm
import subprocess
import os
from   shapely.geometry import Point
import yaml
import shutil
import warnings
import networkx as nx
import re
import copy


warnings.filterwarnings("ignore")

from riverlakenetwork import BurnLakes


In [2]:
# Load general configurations from a YAML file
yaml_file = os.path.abspath('../code/HydroLakes_MERITBasins/HydroLakes_MERITBasins_Config.yaml')
with open(yaml_file, 'r') as f:
    GeneralConfig = yaml.safe_load(f)

# get the path to the merit-basins files
path_out             = GeneralConfig['DomainFolder']
TempFolder           = GeneralConfig['TempFolder']
pfafs                = GeneralConfig['pfafs']
lake_file            = GeneralConfig['LakeFile']
IncludeLakes         = GeneralConfig['IncludeLakes']
riv_path             = GeneralConfig['MERITBasinsPaths']['RivPath']
cat_path             = GeneralConfig['MERITBasinsPaths']['CatPath']
cst_path             = GeneralConfig['MERITBasinsPaths']['CstPath']
riv_file_template    = GeneralConfig['MERITBasinsPaths']['RivFileTemplate']
cat_file_template    = GeneralConfig['MERITBasinsPaths']['CatFileTemplate']
cst_file_template    = GeneralConfig['MERITBasinsPaths']['CstFileTemplate']

In [ ]:
if IncludeLakes:
    #load hydrolakeDataset
    lakeO = gpd.read_file(lake_file) # read the hydrolake dataset
    lakeO = lakeO.loc[lakeO["Hylak_id"] < 200].reset_index(drop=True)
    #lakeO = lakeO.sample(frac=0.10, random_state=42)
    #lakeO = gpd.read_file('lake.gpkg')
    #lakeO = FixHydroLakesForMerit(lakeO) # correct for hydrolakes for merit
    lakeO['x'] = lakeO.centroid.x # add centroid lon
    lakeO['y'] = lakeO.centroid.y # add centroid lat

# loop over the config files:
for pfaf in ['72']:

    # get the list of pfaf for the subdomain
    subdomain = 'pfaf'+pfaf
    
    # create the folder and domain to include
    path_out_temp = TempFolder+'pfaffff'+pfaf+'/'+subdomain+'/'
    if os.path.isdir(path_out_temp):
        try:
            shutil.rmtree(path_out_temp)  # Remove the entire directory and its contents
        except OSError as e:
            print(f"Error: {path_out_temp} - {e}")
    if not os.path.isdir(path_out_temp):
        os.makedirs(path_out_temp)

    # read in the files
    riv, cat = gm.merit_read_file ([pfaf],
                                   riv_path,
                                   riv_file_template,
                                   cat_path,
                                   cat_file_template,
                                   path_cst = cst_path,
                                   cst_file_template = cst_file_template)
    
    # prepare the ntopo for merit
    riv, cat = gm.prepare_ntopo(riv=riv,\
                                riv_cols={'id':'COMID', 'next_id':'NextDownID'},\
                                cat=cat,\
                                cat_cols={'id':'COMID', 'hillslope':'hillslope', 'area' :'unitarea'},\
                                network = 'merit')

In [ ]:
# use DataLoader to
# Example config dictionary
config = {
    "riv": riv,
    "riv_dict": {
        "COMID": {"col":"COMID"},
        "NextDownCOMID": {"col":"NextDownID"},
        "length": {"col":"lengthkm"},
        "uparea": {"col":"uparea","unit":"km2"}
    },
    "cat": cat,
    "cat_dict": {
        "COMID": {"col":"COMID"},
        "unitarea": {"col":"unitarea","unit":"km2"},
    },
    "lake": lakeO,
    "lake_dict": {
        "LakeCOMID": {"col":"Hylak_id"},
        "unitarea": {"col":"Lake_area","unit":"km2"}
    }
}

bl = BurnLakes(config)


In [ ]:
bl.cat.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/cat1.gpkg")
bl.riv.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/riv1.gpkg")
bl.lake.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/lake1.gpkg")

In [ ]:
print(bl.riv[bl.riv["COMID"] == 72045904])

In [ ]:
lakeO

In [ ]:
len([72080897, 72080899])